In [5]:
import os
import io
import mimetypes
from minio import Minio
from minio.error import S3Error

ENDPOINT = "objstore1584api.superdata.vn"
ACCESS_KEY = "pkR4fkf3OMTRjLjseA0k"
SECRET_KEY = "GHxm0g6s87MoLmx2eGlni1x22gkh9FUVOAUJfPt7"

BUCKET = "fileserver-lk"
FOLDER_PREFIX = "productset-images/"   # <-- new folder
SECURE = True                          # True for https, False for http


def ensure_folder_marker(client: Minio, bucket: str, prefix: str) -> None:
    if not prefix.endswith("/"):
        prefix += "/"
    data = io.BytesIO(b"")  # file-like object with .read()
    client.put_object(bucket, prefix, data=data, length=0)


def upload_file(client: Minio, bucket: str, prefix: str, local_path: str, object_filename: str | None = None) -> str:
    """
    Upload one file into prefix. Returns the full object name.
    """
    if not prefix.endswith("/"):
        prefix += "/"

    name = object_filename or os.path.basename(local_path)
    object_name = prefix + name

    content_type, _ = mimetypes.guess_type(local_path)
    content_type = content_type or "application/octet-stream"

    client.fput_object(
        bucket_name=bucket,
        object_name=object_name,
        file_path=local_path,
        content_type=content_type,
    )
    return object_name


def main():
    client = Minio(
        endpoint=ENDPOINT,
        access_key=ACCESS_KEY,
        secret_key=SECRET_KEY,
        secure=SECURE,
    )

    # Sanity: bucket must exist
    if not client.bucket_exists(BUCKET):
        raise RuntimeError(f"Bucket does not exist: {BUCKET}")

    # 1) "Create folder"
    ensure_folder_marker(client, BUCKET, FOLDER_PREFIX)
    print(f"✅ Created folder marker: {FOLDER_PREFIX}")

    # 2) Upload images (add as many paths as you want)
    image_paths = [
        r"E:\temp\Projects\PDFToSQL\Data\NKN-CFK729.png",
    ]

    for p in image_paths:
        obj = upload_file(client, BUCKET, FOLDER_PREFIX, p)
        print("✅ Uploaded:", obj)

        # Verify it exists
        stat = client.stat_object(BUCKET, obj)
        print("   ↳ Verified size:", stat.size, "content-type:", stat.content_type)


if __name__ == "__main__":
    try:
        main()
    except S3Error as e:
        print("❌ MinIO/S3 error:", e)
    except Exception as e:
        print("❌ Error:", e)


✅ Created folder marker: productset-images/
✅ Uploaded: productset-images/NKN-CFK729.png
   ↳ Verified size: 92207 content-type: image/png


### __CHECK STRUCTURE OF S3__

In [7]:
from minio import Minio

client = Minio(
    endpoint="objstore1584api.superdata.vn",
    access_key="pkR4fkf3OMTRjLjseA0k",
    secret_key="GHxm0g6s87MoLmx2eGlni1x22gkh9FUVOAUJfPt7",
    secure=True,   # change to False ONLY if it's http
)

bucket = "fileserver-lk"
prefix = "productset-image"

print("Full structure:")
for obj in client.list_objects(bucket, prefix=prefix, recursive=True):
    print(obj.object_name)


Full structure:
productset-images/
productset-images/NKN-CFK729.png


In [8]:
from minio import Minio

client = Minio(
    "objstore1584api.superdata.vn",
    access_key="pkR4fkf3OMTRjLjseA0k",
    secret_key="GHxm0g6s87MoLmx2eGlni1x22gkh9FUVOAUJfPt7",
    secure=True,
)

bucket = "fileserver-lk"

print("Top-level folders:")
for obj in client.list_objects(bucket, prefix="", recursive=False):
    # In recursive=False mode, MinIO returns "directory-like" entries ending with "/"
    name = obj.object_name
    if name.endswith("/"):
        print(name)


Top-level folders:
documents/
files/
garment-images/
images-of-topic/
images/
productset-images/
purchasing-images/
qms-images/
ws-images/
